In [ ]:
#!conda create -n ag python=3.10
#!conda activate ag
#!conda install -c conda-forge mamba
#!mamba install -c conda-forge -c pytorch -c nvidia autogluon "pytorch=*=*cuda*"
#!mamba install -c conda-forge "ray-tune >=2.6.3,<2.7" "ray-default >=2.6.3,<2.7"  # install ray for faster training

In [2]:
#!pip install autogluon==1.0.0
#!pip install --upgrade numpy pandas scipy
#!pip install numpy==1.26.4
#!pip install pyJoules
#!pip install mxnet-cu110
#!pip install jedi
#!pip install setuptools
#!pip install scikit-learn==1.3.0
#!pip install pandas==2.0.0
#!pip install fsspec==2023.1.0
#!pip install torch==2.0.1+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#cls
# !pip install cudatoolkit


  Using cached autogluon-1.0.0-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon.core-1.0.0-py3-none-any.whl.metadata (13 kB)
  Using cached autogluon.features-1.0.0-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon.tabular-1.0.0-py3-none-any.whl.metadata (14 kB)
  Using cached autogluon.multimodal-1.0.0-py3-none-any.whl.metadata (14 kB)
  Using cached autogluon.timeseries-1.0.0-py3-none-any.whl.metadata (13 kB)
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 819.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
  Using cached autogluon.commo

In [2]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.device_count())  # Should be > 0

True
1


In [8]:
import pandas as pd
import numpy as np
import logging
import os
import time
from sklearn.feature_selection import mutual_info_classif, VarianceThreshold
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Deep Learning Part

In [10]:
FILENAME = "XAPI"
DATA_PATH = "xapi.csv"
TARGET = "Class" 
KFOLD=10    

# Load dataset
df = pd.read_csv(DATA_PATH)
df.columns = df.columns.str.replace(' ', '')

# Separate features and target
X = df.drop(columns=[TARGET])
y = df[TARGET]

# Create a new DataFrame with the selected features and selected class
df_selected = X.copy()
df_selected[TARGET] = y

for gpu_available in [0, 1]:
    for validation_type in ['kfold']:
        start_time = time.time()

        train_data = df_selected
        test_data = df_selected
        
        path = f"GPU_{gpu_available}_{FILENAME}_DL_VALIDATION_{validation_type}"

        # Create the dir if not exist
        os.makedirs(path, exist_ok=True)
        predictor = TabularPredictor(label=TARGET,  path=path, problem_type="multiclass")
        
        if(gpu_available):
            predictor.fit(train_data, num_bag_folds=KFOLD, verbosity=2, num_gpus=1, excluded_model_types= ['RF', 'KNN', 'GBM','XGB','CAT','XT','LR'], presets="best_quality")
        else:
            predictor.fit(train_data, num_bag_folds=KFOLD, verbosity=2, excluded_model_types= ['RF', 'KNN', 'GBM','XGB','CAT','XT','LR'], presets="best_quality")
        
        # Test data evaluaton
        y_pred = predictor.predict(test_data)
        y_true = test_data[TARGET]
        y_prob = predictor.predict_proba(test_data)

        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')
        f1 = f1_score(y_true, y_pred, average='weighted')
        roc_auc = roc_auc_score(pd.get_dummies(y_true), y_prob, average='weighted', multi_class='ovr')

        df1 = pd.DataFrame({
            'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC'],
            'Score': [accuracy, precision, recall, f1, roc_auc]
        })
      
        end_time = time.time()
        execution_time_minutes = (end_time - start_time) / 60

        filename = f"GPU_{gpu_available}_{FILENAME}_DL_VALIDATION_{validation_type}_TIME_{execution_time_minutes:.2f}_EMISSION_{emissions_filtering:.6f}.csv"

        print(filename)
        df1.to_csv(filename, index=False)



[codecarbon INFO @ 09:49:56] [setup] RAM Tracking...
[codecarbon INFO @ 09:49:56] [setup] GPU Tracking...
[codecarbon INFO @ 09:49:56] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 09:49:56] [setup] CPU Tracking...
[codecarbon WARNING @ 09:49:56] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 09:49:57] Energy consumed for RAM : 0.028272 kWh. RAM Power : 23.979151725769043 W
[codecarbon INFO @ 09:49:57] Energy consumed for all GPUs : 0.088731 kWh. Total GPU Power : 25.102485227353 W
[codecarbon INFO @ 09:49:57] Energy consumed for all CPUs : 0.061940 kWh. Total CPU Power : 52.5 W
[codecarbon INFO @ 09:49:57] 0.178943 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:49:57] CPU Model on constant consumption mode: AMD Ryzen 9 5950X 16-Core Processor
[codecarbon INFO @ 09:49:57] >>> Tracker's metadata:
[codecarbon INFO @ 09:49:57]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 09:49:57]   Python version: 3.11.4
[codec

GPU_0_XAPI_DL_VALIDATION_7525_TIME_24.15_EMISSION_0.008908.csv


Leaderboard on holdout data (DyStack):
                          model  score_holdout  score_val eval_metric  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     NeuralNetTorch_r14_BAG_L2       0.851852   0.868545    accuracy        9.102697       3.823455  195.211071                 0.268126                0.242472           2.647561            2       True         57
1    NeuralNetTorch_r185_BAG_L1       0.833333   0.856808    accuracy        0.163827       0.068167    4.276431                 0.163827                0.068167           4.276431            1       True         28
2    NeuralNetFastAI_r88_BAG_L2       0.833333   0.880282    accuracy        9.050313       3.680864  198.560524                 0.215743                0.099881           5.997014            2       True         70
3     NeuralNetTorch_r31_BAG_L2       0.833333   0.868545    accuracy        9.119660       3.869

GPU_0_XAPI_DL_VALIDATION_kfold_TIME_19.68_EMISSION_0.016183.csv


Leaderboard on holdout data (DyStack):
                          model  score_holdout  score_val eval_metric  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   NeuralNetFastAI_r191_BAG_L2          0.875   0.846875    accuracy        8.391735       3.430303  173.245078                 0.187608                0.096799           3.606311            2       True         49
1     NeuralNetTorch_r36_BAG_L2          0.875   0.850000    accuracy        8.434519       3.528975  172.173202                 0.230392                0.195471           2.534435            2       True         84
2     NeuralNetTorch_r19_BAG_L1          0.850   0.815625    accuracy        0.112734       0.066202    2.392196                 0.112734                0.066202           2.392196            1       True         42
3     NeuralNetTorch_r87_BAG_L1          0.850   0.803125    accuracy        0.126568       0.063

GPU_1_XAPI_DL_VALIDATION_7525_TIME_31.06_EMISSION_0.027193.csv


Leaderboard on holdout data (DyStack):
                          model  score_holdout  score_val eval_metric  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     NeuralNetTorch_r14_BAG_L2       0.851852   0.868545    accuracy        8.410296       4.020958  192.335054                 0.251476                0.287229           2.520400            2       True         57
1    NeuralNetTorch_r185_BAG_L1       0.833333   0.856808    accuracy        0.144529       0.069818    4.134268                 0.144529                0.069818           4.134268            1       True         28
2    NeuralNetFastAI_r88_BAG_L2       0.833333   0.880282    accuracy        8.374794       3.833240  195.697948                 0.215975                0.099510           5.883293            2       True         70
3     NeuralNetTorch_r31_BAG_L2       0.833333   0.868545    accuracy        8.419685       3.986

GPU_1_XAPI_DL_VALIDATION_kfold_TIME_57.07_EMISSION_0.047947.csv


In [ ]:
dfl